## Join the Duet Server the Data Owner 1 connected to

In [1]:
import syft as sy

In [2]:
duet1 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


## Join the Duet Server the Data Owner 2 connected to

In [ ]:
duet2 = sy.join_duet(loopback=True)

## Get data from Duet Server

In [ ]:
X_Train_Owner1_ptr = duet1.store[0]
y_Train_Owner1_ptr = duet1.store[1]
X_Test_Owner1_ptr = duet1.store[2]
y_Test_Owner1_ptr = duet1.store[3]

print(X_Train_Owner1_ptr.tags)
print(X_Train_Owner1_ptr.description)

In [ ]:
duet2.store.pandas

In [ ]:
X_Train_Owner2_ptr = duet2.store[0]
y_Train_Owner2_ptr = duet2.store[1]
X_Test_Owner2_ptr = duet2.store[2]
y_Test_Owner2_ptr = duet2.store[3]

print(X_Train_Owner2_ptr.tags)
print(X_Train_Owner2_ptr.description)

## Build a Base Classifier Model

In [ ]:
class SyNet(sy.Module):
    def __init__(self,input_channels,output_channels,torch_ref):
        super(SyNet,self).__init__(torch_ref=torch_ref)
        #Our network:
        # Linear1->relu->Batchnorm->Linear2->relu->Batchnorm->Dropout->Linear3->output
        # Softmax is added in the predict function
        #This applies Linear transformation to input data. 
        self.fc1 = self.torch_ref.nn.Linear(input_channels,int(1.5*input_channels))
        self.fc2 = self.torch_ref.nn.Linear(int(1.5*input_channels),int(1.5*input_channels))
        self.fc3 = self.torch_ref.nn.Linear(int(1.5*input_channels),output_channels)
        
        self.relu = self.torch_ref.nn.ReLU()
        self.dropout = self.torch_ref.nn.Dropout(p=0.1)
        self.batchnorm1 = self.torch_ref.nn.BatchNorm1d(int(1.5*input_channels))
        self.batchnorm2 = self.torch_ref.nn.BatchNorm1d(int(1.5*input_channels))
        self.sigmoid = self.torch_ref.nn.Sigmoid()
    #This must be implemented
    def forward(self,x):
        x = self.relu(self.fc1(x))
        x = self.batchnorm1(x)
        x = self.relu(self.fc2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x
    def predict(self,x):
        output = self.forward(x)
        prediction = self.torch_ref.argmax(output,1)
        return prediction

In [ ]:
def classifier_train(epochs, model, optimizer, X_ptr,y_ptr,criterion,torch_ref):
    losses = []
    for i in range(epochs):
        #Precit the output for Given input
        y_pred_ptr = model.forward(X_ptr)
        #Compute Cross entropy loss
        loss = criterion(y_pred_ptr,y_ptr)
        loss_item = loss.item()
        #Request to get the loss value
        loss_value = loss_item.get(
            reason="To evaluate training progress",
            request_block=True,
            timeout_secs=5,
        )
        #Add loss to the list
        losses.append(loss_value)
        #Print loss
        if i%50==0:
            print("Epoch:",i," Loss:",loss_value)
        
        #Clear the previous gradients
        optimizer.zero_grad()
        #Compute gradients
        loss.backward()
        #Adjust weights
        optimizer.step()
    return losses

In [ ]:
import torch as base_torch         
import numpy as np

In [ ]:
# Base Classifier Model
X_Train_Owner1_shape = X_Train_Owner1_ptr.shape.get(
            reason="To evaluate training progress",
            request_block=True,
            timeout_secs=5,
        )
input_channels = X_Train_Owner1_shape[1]
output_channels = 2
base_model = SyNet(input_channels,output_channels,base_torch)

## Send copy of model to Data Owner1and train them remotely

In [ ]:
remote_model1 = base_model.send(duet1)

remote_torch1 = duet1.torch
params = remote_model1.parameters()
#Define loss criterion
criterion = remote_torch1.nn.CrossEntropyLoss()
#Define the optimizer
optimizer = remote_torch1.optim.Adam(params=params, lr=0.001)
#Number of epochs
epochs = 1000

# Training
losses = classifier_train(epochs, remote_model1, optimizer, X_Train_Owner1_ptr, y_Train_Owner1_ptr,criterion, remote_torch1)

## Send copy of model to Data Owner2and train them remotely

In [ ]:
remote_model2 = base_model.send(duet2)

remote_torch2 = duet2.torch
params = remote_model2.parameters()
#Define loss criterion
criterion = remote_torch2.nn.CrossEntropyLoss()
#Define the optimizer
optimizer = remote_torch2.optim.Adam(params=params, lr=0.001)
#Number of epochs
epochs = 1000

# Training
losses = classifier_train(epochs, remote_model2, optimizer, X_Train_Owner2_ptr, y_Train_Owner2_ptr,criterion, remote_torch2)

## Averaging Model Updates

In [ ]:
from collections import OrderedDict

In [ ]:
param1 = remote_model1.parameters().get(request_block=True)
param2 = remote_model2.parameters().get(request_block=True)

print("Base Model parameters:")
print(base_model.parameters())
print()

print("Remote model1 parameters:")
print(param1)
print()

print("Remote model2 parameters:")
print(param2)

In [ ]:
remote_model1_updates = remote_model1.get(
    request_block=True
).state_dict()

print(remote_model1_updates)

remote_model2_updates = remote_model2.get(
    request_block=True
).state_dict()

print(remote_model2_updates)

In [ ]:
# Get Average Model
avg_updates = OrderedDict()
for key in remote_model1_updates.keys():
    avg_updates[key] = (remote_model1_updates[key] + remote_model2_updates[key]) / 2

    
combined_model = SyNet(input_channels,output_channels,base_torch)

combined_model.load_state_dict(avg_updates)

## Test the prediction of combined model

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

german = pd.read_csv("./data/german_data_2.csv")  

In [ ]:
test_size = 0.20

processed_data = None
categorical = None
label_encoders = {}

def preprocessing(dataset, data, test_size):
    """
    Preprocess dataset

    Parameters
    ----------
    data: DataFrame
        Pandas dataframe containing German dataset.
    """
    
    global processed_data
    global categorical
    global label_encoders

    # Reset global variables
    
    processed_data = None
    categorical = None
    label_encoders = {}


    if dataset == "German":
        # Drop savings account and checkings account columns as they contain a lot
        # of NaN values and may not always be available in real life scenarios
        data = data.drop(columns = ['Saving accounts', 'Checking account'])
        
    dat_dict = data.to_dict()
    new_dat_dict = {}

    # rename columns(Make them lowercase and snakecase)
    for key, value in dat_dict.items():
        newKey = key
        if type(key) == str:
            newKey = newKey.lower().replace(' ', '_')
        # if newKey != key:
        new_dat_dict[newKey] = dat_dict[key]
    del dat_dict

    data = pd.DataFrame.from_dict(new_dat_dict)
    del new_dat_dict


    # print(data.describe())
    # print(data.describe(include='O'))

    cols = data.columns
    num_cols = data._get_numeric_data().columns
    categorical = list(set(cols) - set(num_cols))

    # Drop null rows
    data = data.dropna()

    # Encode text columns to number values
    for category in categorical:
        le = LabelEncoder()
        data[category] = le.fit_transform(data[category])
        label_encoders[category] = le

    for col in data.columns:
        if(col not in categorical):
            data[col] = (data[col].astype('float') - np.mean(data[col].astype('float')))/np.std(data[col].astype('float'))

    # print(data.describe())
    # print(data.describe(include='O'))

    processed_data = data

    # Get Training parameters
    if dataset == "German":
        target_col = data.columns[-1]
        x = data.drop(columns=target_col, axis=1)
        y = data[target_col].astype('int')
    elif dataset == "Australian":
        x = data.drop(14, axis=1)
        y = data[14].astype('int')
    elif dataset == "Japanese":
        x = data.drop(15, axis=1)
        y = data[15].astype('int')
    elif dataset == "Taiwan":
        x = data.drop('default_payment_next_month', axis=1)
        y = data['default_payment_next_month'].astype('int')
    elif dataset == "Polish":
        x = data.drop('class', axis=1)
        y = data['class'].astype('int')


    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = test_size)
    x_train = pd.DataFrame(x_train)
    y_train = pd.DataFrame(y_train)
    y_test = pd.DataFrame(y_test)

    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    
    y_train = y_train[y_train.columns[0]].to_numpy()
    y_test = y_test[y_test.columns[0]].to_numpy()

    return (x_train, x_test, y_train, y_test)

X_train, X_test, y_train, y_test = preprocessing("German", german, test_size)

In [ ]:
# Trnasform the input to tensor
X_test_tensor = base_torch.FloatTensor(X_test)
y_test_tensor = base_torch.tensor(y_test,dtype=base_torch.long)

In [ ]:
# Calcualte Accuracy
from sklearn.metrics import accuracy_score

def accuracy(model,X,y):
    print(accuracy_score(model.predict(X),y))

In [ ]:
accuracy(combined_model,X_test_tensor,y_test_tensor)